In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from math import sqrt
from tqdm.notebook import tqdm
import warnings
import os

# Suppress specific warnings
warnings.simplefilter('ignore', UserWarning)

In [2]:
# Create the 'prophet' directory if it doesn't exist
if not os.path.exists('prophet'):
    os.makedirs('prophet')

In [3]:
# Load the data
data = pd.read_csv('/kaggle/input/inventory/inventory.csv')

In [4]:
# Filter data based on 'Material' Type and 'BFP' being 0
data = data[(data['Type'] == 'Material') & (data['BFP'] == 0)].copy()

# Convert the 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Set the 'Date' column as the index
data.set_index('Date', inplace=True)

In [5]:
# Get unique material codes
material_codes = data['Material Code'].unique()

# Create a dictionary to hold DataFrames for each material code
material_data = {material: data[data['Material Code'] == material] for material in material_codes}

In [6]:
# Initialize a DataFrame to store the results
prophet_results = pd.DataFrame(columns=['Material Code', 'RMSE'])

In [7]:
# Hyperparameter tuning for each material code
for material, df in tqdm(material_data.items(), total=len(material_data), desc="Processing Material Codes"):
    # Drop missing values
    df = df.dropna()
    df.sort_values('Date', inplace=True)
    if df.empty:
        print(f"No data available for material {material}")
        continue

    # Split data into training and testing sets
    train_size = int(len(df) * 0.8)
    train, test = df.iloc[:train_size], df.iloc[train_size:]

    # Prepare data for Prophet
    train_prophet = train.reset_index().rename(columns={'Date': 'ds', 'Material Issued': 'y'})
    test_prophet = test.reset_index().rename(columns={'Date': 'ds', 'Material Issued': 'y'})

    try:
        # Initialize and fit the Prophet model
        model = Prophet()
        model.fit(train_prophet)

        # Forecast the next steps
        future = model.make_future_dataframe(periods=len(test))
        forecast = model.predict(future)

        # Calculate RMSE
        if len(test_prophet['y']) == len(forecast['yhat'][-len(test_prophet):]):
            rmse = sqrt(mean_squared_error(test_prophet['y'], forecast['yhat'][-len(test_prophet):]))

            # Store results
            result_dict = {
                'Material Code': material,
                'RMSE': rmse
            }

            # Print result for current material
            print(result_dict)

            # Append the results to the DataFrame
            prophet_results = pd.concat([prophet_results, pd.DataFrame([result_dict])], ignore_index=True)

            # Plot the results and save the plot
            plt.figure(figsize=(10, 6))
            plt.plot(train_prophet['ds'], train_prophet['y'], label='Train')
            plt.plot(test_prophet['ds'], test_prophet['y'], label='Test')
            plt.plot(forecast['ds'], forecast['yhat'], label='Forecast', linestyle='--')
            plt.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], color='pink', alpha=0.3)
            plt.title(f'Material Code: {material}')
            plt.xlabel('Date')
            plt.ylabel('Material Issued')
            plt.legend()

            # Save the plot to the 'prophet' directory
            plot_filename = f'prophet/material_{material}.png'
            plt.savefig(plot_filename)
            plt.close()  # Close the plot to free up memory
        else:
            print(f"Mismatch in lengths for material {material}")

    except Exception as e:
        print(f"Error fitting Prophet model for material {material}: {e}")
        continue


Processing Material Codes:   0%|          | 0/642 [00:00<?, ?it/s]

16:41:53 - cmdstanpy - INFO - Chain [1] start processing
16:41:53 - cmdstanpy - INFO - Chain [1] done processing
<ipython-input-7-f435772ce761>:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  prophet_results = pd.concat([prophet_results, pd.DataFrame([result_dict])], ignore_index=True)


{'Material Code': 10000033, 'RMSE': 2.370703370498499}


16:41:54 - cmdstanpy - INFO - Chain [1] start processing
16:41:54 - cmdstanpy - INFO - Chain [1] done processing


{'Material Code': 10000304, 'RMSE': 11.646745031476849}


...

{'Material Code': 10130819, 'RMSE': 0.07954236520706078}
{'Material Code': 10130952, 'RMSE': 0.0}
{'Material Code': 10131743, 'RMSE': 0.0}
{'Material Code': 10131745, 'RMSE': 0.0}


16:46:34 - cmdstanpy - INFO - Chain [1] start processing
16:46:34 - cmdstanpy - INFO - Chain [1] done processing


{'Material Code': 10131757, 'RMSE': 1.4942682422262337}
{'Material Code': 10132190, 'RMSE': 0.0}


16:46:35 - cmdstanpy - INFO - Chain [1] start processing
16:46:35 - cmdstanpy - INFO - Chain [1] done processing


{'Material Code': 10132193, 'RMSE': 1.6895801849145813}
{'Material Code': 10132329, 'RMSE': 0.0}
{'Material Code': 10133474, 'RMSE': 0.0}
{'Material Code': 10133599, 'RMSE': 0.0}
{'Material Code': 10134880, 'RMSE': 0.08333333333333333}
{'Material Code': 10135324, 'RMSE': 0.0}
{'Material Code': 10135325, 'RMSE': 0.0}
{'Material Code': 10135326, 'RMSE': 0.0}
{'Material Code': 10135384, 'RMSE': 0.0}
{'Material Code': 10136723, 'RMSE': 0.0}
{'Material Code': 10136724, 'RMSE': 0.0}
{'Material Code': 10136725, 'RMSE': 0.0}
{'Material Code': 10136726, 'RMSE': 0.0}
{'Material Code': 10136727, 'RMSE': 0.0}
{'Material Code': 10136728, 'RMSE': 0.0}
{'Material Code': 10136729, 'RMSE': 0.0}
{'Material Code': 10136730, 'RMSE': 0.0}


In [8]:
# Display the Prophet model metrics DataFrame
display(prophet_results)

# Optionally, save the DataFrame to a CSV file
prophet_results.to_csv('prophet_results.csv', index=False)

,Material Code,RMSE
0,10000033,2.370703
1,10000304,11.646745
2,10000314,2.088043
3,10000316,1.943928
4,10000318,20.504995
...,...,...
637,10136726,0.000000
638,10136727,0.000000
639,10136728,0.000000
640,10136729,0.000000
